## Import Libraries

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


In [ ]:
print(tf.__version__)

In [ ]:
def get_files(base_dir, target_dir):
    count = 0
    path = get_path(base_dir, target_dir)
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            count+=len(glob.glob(os.path.join(dirname, filename)))
        return path, count
    
def get_path(base_dir, target_dir):
    path = os.path.join(base_dir,target_dir)
    return path

## Directory Setup

In [ ]:
base_dir = '/kaggle/input/cassava-leaf-disease-classification'
train_dir = 'train_images'
labels_file = 'train.csv'
test_dir = 'test_images'
json_file = 'label_num_to_disease_map.json'

train_path, train_count = get_files(base_dir,train_dir)
test_path, test_count = get_files(base_dir,test_dir)

with open(get_path(base_dir,json_file)) as f:
    class_names = json.load(f)
    class_dict = pd.Series(class_names.values()).to_dict()
    f.close()

data = pd.read_csv(get_path(base_dir, labels_file))
data['class_name'] = data.label.map(class_dict)

print("No of Train Images: {}".format(train_count))
print("No of Test Images: {}".format(test_count))
print("No of Classes: {}".format(len(class_dict)))
print("Classes:")
for v in class_dict.values():
    print(" \u2022 {}".format(v))

In [ ]:
data.info()

In [ ]:
data['class_name'].value_counts().plot(kind='bar')

In [ ]:
for dirname, _, filenames in os.walk(train_path):
    for filename in filenames:
        image = cv2.imread(os.path.join(train_path, filename))
        image_size = image.shape
        break

image_size

## Visualization

In [ ]:
def visualize_img(images):
    fig = plt.figure(figsize=(20, 15))
    for i,a in enumerate(images):
        fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
        path = get_path(train_path, a)
        img = cv2.imread(path)
        plt.imshow(img)
        plt.title(data[data.image_id == a].class_name.values[0])
    
fig = plt.figure(figsize=(15, 15))
p=0
for i in range(5):
    images = data[data.label == i].image_id
    images = np.random.choice(images , 4)
    visualize_img(images)

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

## Example of Image Augmentation

In [ ]:
BATCH_SIZE = 64
IMG_SHAPE  = 224

train_image_gen = ImageDataGenerator(rescale=1./255,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     brightness_range=[0.2,1.0],
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     vertical_flip=True,
                                     fill_mode='nearest')

train_gen = train_image_gen.flow_from_dataframe(data,
                                          directory=train_path,
                                          x_col='image_id',
                                          y_col='class_name',
                                          class_mode='categorical',
                                          batch_size=BATCH_SIZE,
                                          shuffle=True,
                                          target_size=(IMG_SHAPE,IMG_SHAPE))

In [ ]:
augmented_images = [train_gen[0][0][1] for i in range(5)]
plotImages(augmented_images)

## Data Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

train, val = train_test_split(data, test_size = 0.25, random_state=42)

lb = LabelBinarizer()
lb.fit(data.label)

train_lb = lb.transform(train.label)
val_lb = lb.transform(val.label)

train = train.reset_index().drop(labels='index', axis=1)
y_train = pd.DataFrame(train_lb).add_prefix('label_')

val = val.reset_index().drop(labels='index', axis=1)
y_val = pd.DataFrame(val_lb).add_prefix('label_')

train = pd.concat([train, y_train], axis=1)
val = pd.concat([val, y_val], axis=1)

print("Training set has {} samples".format(train.shape[0]))
print("Validation set has {} samples".format(val.shape[0]))

### Helper Functions

In [ ]:
BATCH_SIZE = 32
IMG_SHAPE  = 224
EPOCHS = 30

def gen():
    train_image_gen = ImageDataGenerator(rescale=1./255,
                                         width_shift_range=0.1,
                                         height_shift_range=0.1,
                                         brightness_range=[0.2,1.0],
                                         zoom_range=0.2,
                                         horizontal_flip=True,
                                         vertical_flip=True,
                                         fill_mode='nearest')

    train_gen = train_image_gen.flow_from_dataframe(train,
                                              directory=train_path,
                                              x_col='image_id',
                                              y_col=[f'label_{x}' for x in np.arange(5)],
                                              class_mode='raw',
                                              batch_size=BATCH_SIZE,
                                              shuffle=True,
                                              target_size=(IMG_SHAPE,IMG_SHAPE))


    val_image_gen = ImageDataGenerator(rescale=1./255)

    val_gen = val_image_gen.flow_from_dataframe(val,
                                              directory=train_path,
                                              x_col='image_id',
                                              y_col= [f'label_{x}' for x in np.arange(5)],
                                              class_mode='raw',
                                              batch_size=BATCH_SIZE,
                                              target_size=(IMG_SHAPE,IMG_SHAPE))
    return train_gen, val_gen


In [ ]:
def plot(history):

    training_accuracy = history.history['accuracy']
    validation_accuracy = history.history['val_accuracy']

    training_loss = history.history['loss']
    validation_loss = history.history['val_loss']

    epochs_range=range(len(training_accuracy))

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, training_accuracy, label='Training Accuracy')
    plt.plot(epochs_range, validation_accuracy, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, training_loss, label='Training Loss')
    plt.plot(epochs_range, validation_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

In [ ]:
from PIL import Image
def predict(image_path, model):
    im = Image.open(image_path)
    test_image = np.asarray(im)
    processed_test_image = process_image(test_image)
    processed_test_image = np.expand_dims(processed_test_image, axis = 0)
    
    ps = model.predict(processed_test_image)
    return ps
    
def process_image(image):
    image = tf.cast(image , tf.float32)
    image = tf.image.resize(image , (224 , 224))
    image = image/255
    image = image.numpy()
    return image


In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced', np.unique(train['label']), train.label)
class_weights = dict(enumerate(class_weights))
class_weights

## Modeling

1. **VGG-16 Model** 

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
tf.keras.backend.clear_session()

base = VGG16(weights = 'imagenet' , include_top=False, input_shape=(IMG_SHAPE, IMG_SHAPE, 3))   

vgg16_model = Sequential()
vgg16_model.add(base)
vgg16_model.add(GlobalAveragePooling2D())
vgg16_model.add(Dropout(0.5))
vgg16_model.add(BatchNormalization())
vgg16_model.add(Dense(128, activation='relu'))
vgg16_model.add(Dropout(0.5))
vgg16_model.add(Dense(5, activation='softmax'))

vgg16_model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(vgg16_model, to_file='vgg16_model.png', show_shapes=True, show_layer_names=True)

In [ ]:
train_gen, val_gen = gen()

optm = Adam(lr=0.0001)
vgg16_model.compile(loss='categorical_crossentropy', optimizer=optm, 
                  metrics=['accuracy'])

EarlyStopping = EarlyStopping(monitor='val_loss',
                              min_delta=.0001,
                              patience=5,
                              verbose=1,
                              mode='auto',
                              baseline=None,
                              restore_best_weights=True)

model_save = ModelCheckpoint('./vgg16.h5',
                             save_best_only = True, 
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)


vgg_history = vgg16_model.fit_generator(train_gen,
                                    steps_per_epoch = train_gen.samples // BATCH_SIZE,
                                    epochs = 30,
                                    validation_data = val_gen,
                                    validation_steps = val_gen.samples // BATCH_SIZE,
                                    callbacks=[EarlyStopping, model_save])

In [ ]:
plot(vgg_history)

In [ ]:
test_images = os.listdir(test_path)    
for image in test_images:
    vgg16_pred = predict(os.path.join(test_path, image) , vgg16_model)

In [ ]:
vgg16_pred

In [ ]:
np.argmax(vgg16_pred)

In [ ]:
vgg16_preds = vgg16_model.predict(val_gen)
vgg16_preds = np.argmax(vgg16_preds, axis=-1)
org_label = val['label'].astype('int')

matrix=confusion_matrix(org_label, vgg16_preds)
sns.heatmap(matrix,square=True, annot=True, fmt='d', cbar=False,
            xticklabels=['0', '1', '2', '3', '4'],
            yticklabels=['0', '1', '2', '3', '4'])
plt.xlabel('Predicted label')
plt.ylabel('True label');


2. MobileNetV2

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

base = MobileNetV2(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
tf.keras.backend.clear_session()

for layers in base.layers:   
    layers.trainable = True
    
for layer in base.layers[:100]:
    layer.trainable =  False

mobilenet_model = Sequential()
mobilenet_model.add(base)
mobilenet_model.add(GlobalAveragePooling2D())
mobilenet_model.add(BatchNormalization())
mobilenet_model.add(Dense(256, activation='relu'))
mobilenet_model.add(Dropout(0.5))
mobilenet_model.add(BatchNormalization())
mobilenet_model.add(Dense(128, activation='relu'))
mobilenet_model.add(Dropout(0.5))
mobilenet_model.add(Dense(5, activation='softmax'))

mobilenet_model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(mobilenet_model, to_file='mobilenetV2.png', show_shapes=True, show_layer_names=True)

In [ ]:
train_gen, val_gen = gen()

optm = Adam(lr=0.0001)
mobilenet_model.compile(loss='categorical_crossentropy', optimizer=optm, 
                  metrics=['accuracy'])

EarlyStopping = EarlyStopping(monitor='val_loss',
                              min_delta=.0001,
                              patience=3,
                              verbose=1,
                              mode='auto',
                              baseline=None,
                              restore_best_weights=True)

model_save = ModelCheckpoint('./mobilenetV2.h5',
                             save_best_only = True,
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)


mob_history = mobilenet_model.fit(train_gen,
                              steps_per_epoch = train_gen.samples // BATCH_SIZE,
                              epochs = EPOCHS,
                              validation_data = val_gen,
                              validation_steps = val_gen.samples // BATCH_SIZE,
                              callbacks=[EarlyStopping, model_save])

In [ ]:
plot(mob_history)

In [ ]:
print(mob_history.history['val_accuracy'][-4])
print(mob_history.history['val_loss'][-4])

In [ ]:
test_images = os.listdir(test_path)    
for image in test_images:
    mobilenet_pred = predict(os.path.join(test_path, image) , mobilenet_model)

In [ ]:
mobilenet_pred

In [ ]:
np.argmax(mobilenet_pred)

In [ ]:
mob_preds = mobilenet_model.predict(val_gen)
mob_preds = np.argmax(mob_preds, axis=-1)
org_label = val['label'].astype('int')

matrix=confusion_matrix(org_label, mob_preds)
sns.heatmap(matrix,square=True, annot=True, fmt='d', cbar=False,
            xticklabels=['0', '1', '2', '3', '4'],
            yticklabels=['0', '1', '2', '3', '4'])
plt.xlabel('Predicted label')
plt.ylabel('True label');

3. DenseNet169

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
base = DenseNet169(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
tf.keras.backend.clear_session()

for layer in base.layers:   
    layer.trainable = False

densenet_model = Sequential()
densenet_model.add(base)
densenet_model.add(GlobalAveragePooling2D())
densenet_model.add(BatchNormalization())
densenet_model.add(Dense(256, activation='relu'))
densenet_model.add(Dropout(0.5))
densenet_model.add(BatchNormalization())
densenet_model.add(Dense(128, activation='relu'))
densenet_model.add(Dropout(0.5))
densenet_model.add(Dense(5, activation='softmax'))

densenet_model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(densenet_model, to_file='densenet169.png', show_shapes=True, show_layer_names=True)

In [ ]:
train_gen, val_gen = gen()

optm = Adam(lr=0.0001)
densenet_model.compile(loss='categorical_crossentropy', optimizer=optm, 
                  metrics=['accuracy'])

EarlyStopping = EarlyStopping(monitor='val_loss',
                              min_delta=.0001,
                              patience=5,
                              verbose=1,
                              mode='auto',
                              baseline=None,
                              restore_best_weights=True)

model_save = ModelCheckpoint('./densenet.h5',
                             save_best_only = True,
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)


dense_history = densenet_model.fit_generator(train_gen,
                              steps_per_epoch = train_gen.samples // BATCH_SIZE,
                              epochs = EPOCHS,
                              validation_data = val_gen,
                              validation_steps = val_gen.samples // BATCH_SIZE,
                              callbacks=[EarlyStopping, model_save])

In [ ]:
plot(dense_history)

In [ ]:
print(dense_history.history['val_accuracy'][-5])
print(dense_history.history['val_loss'][-5])

In [ ]:
test_images = os.listdir(test_path)    
for image in test_images:
    densenet_pred = predict(os.path.join(test_path, image) , densenet_model)

In [ ]:
densenet_pred

In [ ]:
np.argmax(densenet_pred)

In [ ]:
dense_preds = densenet_model.predict(val_gen)
dense_preds = np.argmax(dense_preds, axis=-1)
org_label = val['label'].astype('int')

matrix=confusion_matrix(org_label, dense_preds)
sns.heatmap(matrix,square=True, annot=True, fmt='d', cbar=False,
            xticklabels=['0', '1', '2', '3', '4'],
            yticklabels=['0', '1', '2', '3', '4'])
plt.xlabel('Predicted label')
plt.ylabel('True label');

## Ensemble

In [ ]:
mobilenet_model = tf.keras.models.load_model("../input/cassava-models/mobilenetV2.h5")
vgg_model = tf.keras.models.load_model("../input/cassava-models/vgg16.h5")
densenet_model = tf.keras.models.load_model("../input/cassava-models/densenet.h5")

In [ ]:
model1_list=[]
model2_list=[]
model3_list = []
predicted_label_list = []
actual_labels=[]

val_images = val.image_id
for i in val_images:
    actual_labels.append(int(val[val.image_id == i].label))   
actual_labels = pd.Series(actual_labels)

for image in val_images:
    model1_list.append(predict(os.path.join(train_path, image), densenet_model))
    model2_list.append(predict(os.path.join(train_path, image), mobilenet_model))
    model3_list.append(predict(os.path.join(train_path, image), vgg_model))

for dense, mob, vgg in zip(model1_list, model2_list, model3_list):
    predicted_label_list.append(np.argmax(dense/np.linalg.norm(dense) + mob/np.linalg.norm(mob) + vgg/np.linalg.norm(vgg)))
    
print(classification_report(actual_labels, predicted_label_list))

In [ ]:
matrix=confusion_matrix(actual_labels, predicted_label_list)
sns.heatmap(matrix,square=True, annot=True, fmt='d', cbar=False,
            xticklabels=['0', '1', '2', '3', '4'],
            yticklabels=['0', '1', '2', '3', '4'])
plt.xlabel('Predicted label')
plt.ylabel('True label')